In [1]:
!pip install nn_builder
# !pip uninstall torch -y
# !pip install torch==1.4.0
from gym.spaces import Discrete
import numpy as np
import gym
import matplotlib.pyplot as plt
from gym import spaces
# from google.colab import files
import torch
import random
from collections import deque
import pandas as pd
from nn_builder.pytorch.CNN import CNN   
from nn_builder.pytorch.RNN import RNN   
import pandas as pd
import seaborn as sns
from torch.distributions import Categorical
import torch.nn.functional as F

^C
ERROR: Operation cancelled by user


ModuleNotFoundError: No module named 'nn_builder'

# HELPER FUNCTIONS

In [ ]:
##### HELPER FUNCTIONS #####
import time
import datetime

def copy_model_over(from_model, to_model):
        """Copies model parameters from from_model to to_model"""
        for to_model, from_model in zip(to_model.parameters(), from_model.parameters()):
            to_model.data.copy_(from_model.data.clone())
            
def plot_results(results, rolling, download=False, filename=""):
    """
    Plotting the results
    :param list_rewards: list of cumulative rewards of all episodes
    :param list_cumulative_steps: list of number of actions of all episodes
    :param algo_name: the name of the learning algorithm
    :param download_plots: "True" if this notebook is running on Google Colab and you want to save the figures to your computer, "False" otherwise
    """
    pd_results = pd.DataFrame()
    for r in results:
        pd_results = pd.concat([pd_results, r], axis=1)
        
    list_agent_names = pd_results.columns.to_list()
    pd_data = pd_results.reset_index()
    bines = [(i) * rolling for i in list(range(201))]
    bines_lab = [(i) * rolling for i in list(range(200))]
    pd_data['binned_episode'] = pd.cut(pd_data['index'], bins=bines, labels=bines_lab)
    pd_data = pd_data.fillna(0)
    
    for agent in list_agent_names:
        sns.lineplot(data=pd_data, x="binned_episode", y=agent, label=agent)
   
    plt.title(filename)
    plt.xlabel('Episode')
    plt.ylabel('Reward')
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.subplots_adjust(0,0,1,1,0,0)
    
    date = datetime.datetime.now().strftime("%Y_%m_%d-%I_%M_%S_%p")
    plt.savefig(filename+ "_figure_" + date + ".pdf", dpi=300, bbox_inches='tight', pad_inches = 0)
    filename = filename + "_results_" + date + ".csv"
    
    if download:
        pd_data.to_csv(filename, header=True, index=False)

# Segment Environment

In [ ]:
class SegmentEnv(object):
    def __init__(self, max_steps, window, segment_length, noise, free_location, exploration_cost, pred_reward):
        self.env_name = "segment"
        self.board_size_x = 1
        self.board_size_y = segment_length
        self.num_state_channels = 2
        self.num_action_channels = 2
        
        self.pred_reward = pred_reward # Final reward for the correct prediction
        self.free_location = free_location
        self.noise = noise
        self.segment_length = segment_length
        self.action_space = (Discrete(self.segment_length), Discrete(self.segment_length))
        self.observation_space = Discrete(self.segment_length)
        self.window = window
        self.max_steps = max_steps # Maximum number of steps in one episode
        self.to_draw = torch.zeros((self.max_steps + 1, 1, self.segment_length, 3))
        self.exploration_cost = exploration_cost # The negative reward for each step
        self.window = window
        self.nA = self.num_action_channels * self.board_size_x * self.board_size_y # Size of action space n*x*y

    def seed(self, seed):
        np.random.seed(seed)
    
    def generate_a_segment(self, length, noise=0.0, free_location=False):
        '''
        Generate randomly a new bi-color segment
        '''
        if free_location:
            l1 = np.random.randint(1, length-1)
            l2 = np.random.randint(1, length-1)
            left = min([l1, l2])
            right = max([l1, l2])+1
            segment = [0]*left + [1]*(right-left) + [0]*(length-right)
        else:
            right = np.random.randint(1, length)
            segment = [[0]*right + [1]*(length-right)] # size [1, length]
#         segment = np.array(segment) + noise * np.random.random(length)
        segment = torch.tensor(segment)
        return segment, torch.tensor(right)

    def reset(self, NEXT=True):
        self.pos = None
        self.num_steps = 0 # Current number of steps in an episode so far
        if NEXT:
            self.curr_img, self.target = self.generate_a_segment(length=self.segment_length, noise=self.noise,
                                                            free_location=self.free_location)

        self.state = torch.zeros((self.num_state_channels, 1, self.segment_length))
#         self.state[2] = torch.ones(1, self.segment_length)
        
#         self.state[3] = (-1) * torch.ones(1, self.segment_length)
        
        return self.state
    
    def observe(self):
        """
          Return the current state
        """
        return self.state
    
    def get_frame(self, t, pred=None):
        true_image = np.zeros((1, self.segment_length, 3)).astype(int)
        true_image[:, self.curr_img == 1, 2] = 255
        true_image[:, self.curr_img == 0, 0] = 255

        segment_plot = np.zeros((1, self.segment_length, 3)).astype(int) + 128
        segment_plot[:, self.state[0] == 1, :] = true_image[:, self.state[0] == 1, :]
        if self.pos is not None:
            segment_plot[:, self.pos, :] = np.clip(segment_plot[:, self.pos, :] + 170, 0, 255)
        if pred is not None:
            segment_plot[:, pred, 1] = np.clip(segment_plot[:, pred, 1] + 170, 0, 255)

        self.to_draw[t, :, :, :] = segment_plot

    def step(self, action):
        # Reveal     
#         print("action=", action)
        if action[0] == 0:
#             print("action=", action)
            self.pos = action[2]
            done = False
            reward = -self.exploration_cost
            
            self.state[0, 0, max(self.pos - self.window, 0):min(self.pos + self.window + 1, self.segment_length)] = torch.ones(
                min(self.pos + self.window + 1, self.segment_length) - max(self.pos - self.window, 0))
            
            self.state[1, 0, max(self.pos - self.window, 0):min(self.pos + self.window + 1, self.segment_length)] = self.curr_img[0, max(
                self.pos - self.window, 0):min(self.pos + self.window + 1, self.segment_length)]
            pred = None

        # Predict
        else:
            # Receive the final reward if the agent predicts correctly
            reward = (self.target == action[2]) * self.pred_reward + (-self.exploration_cost)
            done = self.target == action[2]
            pred = action[2] # Predicted position

#         self.get_frame(int(self.num_steps), pred=pred)
        self.num_steps += 1
    
#         Remaining time = (max_steps - current_step)/max_steps
#         self.state[2] = ((self.max_steps - self.num_steps)/self.max_steps) * self.state[2] 
        
#         #Last action
#         self.state[3] = (-1) * torch.ones(1, self.segment_length)
#         self.state[3, 0, action[2]] = 1
        
        info = {}
        
        return self.state, torch.tensor(reward), torch.tensor(int(done == True)), info

    def render(self, e):
        true_image = np.zeros((1, self.segment_length, 3)).astype(int)
        true_image[:, self.curr_img == 1, 2] = 255
        true_image[:, self.curr_img == 0, 0] = 255

        array_list = [
            np.vstack([s_plot, true_image]) for s_plot in self.to_draw[:self.num_steps]
            ]
        image_list = []
        for a in array_list:
            fig = plt.figure()
            plt.yticks([])
            plt.imshow(a)
            plt.hlines(0.5, -0.5, self.segment_length - 0.5)
            fig.canvas.draw()
            image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
            image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
            plt.close(fig)
            image_list.append(image)
        imageio.mimsave('./{}.gif'.format(e), image_list, fps=1)

    def get_current_obs(self):
        return self.state

    def close(self):
        pass

In [ ]:
right = np.random.randint(1, 10)
segment = [[0]*right + [1]*(10-right)]
segment = torch.tensor(segment)
segment.size()

# Batteship Environment

In [ ]:
class BatteshipEnv(gym.Env):
    def __init__(self, board_size_x, board_size_y, num_small_ships, num_medium_ships, num_large_ships, max_steps):
        """
        Initialize a Battleship environment
        :param board_size: With board_size = n we have a board of n*n
        :param ships_board: Given board with placed ships
        :return BatteshipEnv object
        """
#         super(BatteshipEnv, self).__init__()

        self.board_size_x = board_size_x
        self.board_size_y = board_size_y
        
        self.num_state_channels = 2
        self.num_action_channels = 1
        
        self.env_name = "battleship"
        
        self.EMPTY = 0 
        self.UNKNOWN = 1
        self.LAST_ACTION = 1
        self.ships_board = torch.zeros(self.board_size_x, self.board_size_y)
        self.nA = self.board_size_x * self.board_size_y # Size of action space n*n
     
        self.num_small_ships =  num_small_ships
        self.num_medium_ships = num_medium_ships
        self.num_large_ships = num_large_ships
#         self.observation_space = spaces.Discrete(self.board_size * self.board_size) # Obeservation space
#         self.action_space = spaces.Tuple((spaces.Discrete(self.board_size),spaces.Discrete(self.board_size))) # Action space
#         self.nS = 2*self.board_size * self.board_size
        
        self.max_steps = max_steps
        self.num_steps = 0
            
    def calculate_reward(self, action):
        '''
        Return reward for an action
        :param action: action taken
        :return reward 
        ''' 
        
        x = action[0] # x coordinate of the action
        y = action[1] # y coordinate of the action
        
        if (self.s[0,x,y] != self.UNKNOWN): #Already revealed
            if (self.s[1,x,y] == self.EMPTY): #Punish for repeating hitting empty place
                return torch.tensor(-1.0) #-1 
            else: #Keep hitting the ship
                self.num_hits = self.num_hits + 1
                return torch.tensor(1.0) 
        else: #Not revealed
            if (self.ships_board[x,y] == self.EMPTY): #Empty place
                return torch.tensor(-1.0)
            else: #Found a ship
                self.num_hits = self.num_hits + 1
                return torch.tensor(1.0)
        return torch.tensor(-1.0)
            
    def update_board(self, action):
        """
        Update the playing board
        :param action: action taken
        :return the current state after taking the action
        """
        x = action[0] # x coordinate of the action
        y = action[1] # y coordinate of the action
        
        if self.s[0,x,y] == self.UNKNOWN:
            if self.ships_board[x,y] != self.EMPTY:
                self.s[1,x,y] = self.ships_board[x,y] - 1
            else:
                self.s[1,x,y] = self.EMPTY
        else:
            self.s[1,x,y] = max(self.s[1,x,y]-1,0)
            
        self.s[0,x,y] = 1-self.UNKNOWN
        
#         Remaining time = (max_steps - current_step)/max_steps
#         self.s[2] = ((self.max_steps - self.num_steps)/self.max_steps) * self.s[2] 

#         self.s[2] = (-1) * torch.ones(self.board_size_x, self.board_size_y)  
#         self.s[2, x, y] = 1
        
#         
        
        return self.s

    def step(self, action):
        """ 
          Take a step by executing an action
          :param actionL action taken
          :return next_state: the state after taking the action
          :return reward: reward for taking action from the current state
          :return done: Whether all the ships have been destroyed 
          :return info: none
        """
        reward = self.calculate_reward(action)
        next_state = self.update_board(action)
        if self.num_hits >= self.total_hits:
            done = torch.tensor(1)
        else:
            done = torch.tensor(0)
        info = {}
        self.num_steps = self.num_steps + 1
        return next_state, reward, done, info

    def observe(self):
        """
          Return the current state
        """
        return self.s

    def reset(self):
        """
          Reset the environment
          :return the initial playing board
        """
        self.ships_board = self.EMPTY * torch.ones(self.board_size_x, self.board_size_y)
        self.num_steps = 0
        
        self.s = torch.ones(self.num_state_channels, self.board_size_x, self.board_size_y)  
        self.s[0] = self.UNKNOWN * self.s[0]    # Revealed position
        self.s[1] = self.EMPTY * self.s[1]      # Ship's health
#         self.s[2] = self.s[2]          
        
#         self.s[2] = (-1) * self.s[2]            # Last action
                 
        
        
        num_small_ships = self.num_small_ships
        num_medium_ships = self.num_medium_ships
        num_large_ships = self.num_large_ships

        while num_small_ships>0:
            i = random.randint(0, self.board_size_x-1)
            j = random.randint(0, self.board_size_y-1)
            if self.ships_board[i, j] == 0:
                self.ships_board[i, j] = 1
                num_small_ships -= 1

        while num_medium_ships>0:
            i = random.randint(0, self.board_size_x-1)
            j = random.randint(0, self.board_size_y-1)
            if self.ships_board[i, j] == 0:
                self.ships_board[i, j] = 2
                num_medium_ships -= 1

        while num_large_ships>0:
            i = random.randint(0, self.board_size_x-1)
            j = random.randint(0, self.board_size_y-1)
            if self.ships_board[i, j] == 0:
                self.ships_board[i, j] = 3
                num_large_ships -= 1
        
#         self.ships_board[0, 0] = 3
#         self.ships_board[2, 2] = 2
        
        self.total_hits = torch.sum(self.ships_board)
        self.num_hits = 0
        return self.s

    def render(self, mode='human'):
        """
          Render the current playing board
        """
        for i in range(self.board_size_x):
          print("-------------------------------------------")
          line = ""
          for j in range(self.board_size_y):     
            line += " | "
            if self.s[i,j] == self.EMPTY:
              line += "O"
            elif self.s[i,j] == self.UNKNOWN:
              line += " "
            else:
              line += str(int(self.s[i,j]))
          line += " | "
          print(line)

# REPLAY BUFFER

In [ ]:
class Replay():
    """
    Memory for storing experience 
    """
    def __init__(self):
#         self.buffer = []
        self.buffer = deque(maxlen=100000)

    def __len__(self):
        return len(self.buffer)

    def add(self, state, action, reward, next_state, done):
        """
        Add a new experience to the buffer
        :param state: The state before taking the action
        :param action: action taken
        :param reward: Reward for taking that action
        :param next_state: The state that the agent enters after taking the action
        :param done: Whether the agent finishes the game
        """
        self.buffer.append((state, action, reward, next_state, done))

    def sample(self, batch_size):
        """
          Return a batch of samples from the experience buffer
          :param batch_size: The number of sample that you want to take
          :return the batch of samples but decomposed into lists of states, actions, rewards, next_states 
        """
        states, actions, rewards, next_states, dones = [], [], [], [], []

        # Random samples
        samples = random.sample(self.buffer, batch_size)

        for s in samples:
            state = s[0]
            action = s[1]
            reward = s[2]
            next_state = s[3]
            done = s[4]
            
            states.append(state)
            actions.append(action)
            rewards.append(reward)
            next_states.append(next_state)
            dones.append(done)
        
        return states, actions, rewards, next_states, dones

# BASE AGENT

In [ ]:
class Base_Agent():
    def __init__(self, agent_name, env):
        self.agent_name = agent_name
        self.env = env
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.list_rewards = pd.DataFrame (columns = [agent_name])
        self.epsilon = None
        self.epsilon_decay_rate = None
    def select_action(self, state):
        pass
    
    def train():
        pass
    
    def soft_update_of_target_network(self, local_model, target_model, tau):
        """Updates the target network in the direction of the local network but by taking a step size
        less than one so the target network's parameter values trail the local networks. This helps stabilise training"""
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)
    
    def runner(self, number_of_episodes, training=False):
        print("########## " + self.agent_name + " is running ##########")
        
        for episode in range(number_of_episodes+1):
            ep_reward = 0.0         # Reward for this episode
            done = False          # Whether the game is finished
            loss = 0.0
            self.env.reset()

            # Get the current state
            state = self.env.observe().to(self.device, dtype=torch.float)
            
            while not done and self.env.num_steps<self.env.max_steps:
                with torch.no_grad():
                    # Get and execute the next action for the current state
                    action = self.select_action(state)
                    next_state, reward, done, info = env.step(action)
                    ep_reward = ep_reward + reward.item()
                    
                # Start training once the size of the buffer greater than the batch size
                if training:
                    # Save what the agent just learnt to the experience buffer.
                    self.buffer.add(state, action, reward, next_state, done)
                    loss = self.train()
                    
                state = next_state.to(self.device, dtype=torch.float)
                
            # Print log
            print(f'Agent: {self.agent_name} . Episode {episode}/{number_of_episodes} . Number of steps to finish: {self.env.num_steps} . Loss: {loss} '
                f'Reward: {ep_reward}')
            
            # Epsilon is decayed since the agent is getting more and more knowledge
            if self.epsilon != None and self.epsilon_decay_rate != None:
                print("Epsilon:", self.epsilon)
                self.epsilon = self.epsilon * self.epsilon_decay_rate
#                 self.epsilon = max(self.epsilon * self.epsilon_decay_rate, 0.1) 

            self.list_rewards = self.list_rewards.append({self.agent_name:ep_reward}, ignore_index=True)

        env.close()

# RANDOM AGENT

In [ ]:
class Random_Agent(Base_Agent):
    """
    Model a random agent
    """
    def __init__(self, agent_name, env):
        super().__init__(agent_name, env)        

    def select_action(self, state):
        """
          Return an random action
          :param state: the current state
          :return action: next action to take
        """
        if self.env.env_name == "segment": 
            random_action = torch.tensor([random.randint(0,self.env.num_action_channels-1), random.randint(0,self.env.board_size_x-1), random.randint(0,self.env.board_size_y-1)])
        elif self.env.env_name == "battleship":
            random_action = torch.tensor([random.randint(0,self.env.board_size_x-1), random.randint(0,self.env.board_size_y-1)])
        
        return random_action

# Battleship Baseline

In [ ]:
class Battleship_Baseline(Base_Agent):
    """
    Baseline for Battleship
    """
    def __init__(self, agent_name, env):
        super().__init__(agent_name, env)
        self.last_action = None

    def select_action(self, state):
        """
          Return an random action
          :param state: the current state
          :return action: next action to take
        """
      
        x = random.randint(0,self.env.board_size_x-1)
        y = random.randint(0,self.env.board_size_y-1)
            
        if self.last_action == None:
            x = random.randint(0,self.env.board_size_x-1)
            y = random.randint(0,self.env.board_size_y-1)
            random_action = torch.tensor([x, y])
            self.last_action = random_action
            return random_action
        else:
            if state[1, self.last_action[0], self.last_action[1]] == self.env.EMPTY:
                while state[0, x, y] != self.env.UNKNOWN and state[1, x, y] == self.env.EMPTY:
                    x = random.randint(0,self.env.board_size_x-1)
                    y = random.randint(0,self.env.board_size_y-1)
                random_action = torch.tensor([x, y])
                self.last_action = random_action
                return random_action
            else:
                repeat_action = self.last_action
                return repeat_action

# DDQN AGENT

In [ ]:
class DDQN(Base_Agent):
    """
    Model a DQN agent
    """
    def __init__(self, agent_name, env, epsilon, epsilon_decay_rate, gamma, batch_size):
        super().__init__(agent_name, env)        
        self.epsilon = epsilon # For deciding exploitation or exploration
        self.epsilon_decay_rate = epsilon_decay_rate # Epsilon is decayed after each episode with a fixed rate
        self.gamma = gamma # The weight for future rewards
        self.batch_size = batch_size
        
        self.buffer = Replay()             # Experience Buffer
        
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        
        self.main_dqn = CNN(input_dim=(self.env.num_state_channels, self.env.board_size_x, self.env.board_size_y), 
                            layers_info=[
                            ["conv", self.env.num_state_channels, 2, 1, 1], 
                            ["linear", 64],
                            ["linear", 16],
                            ["linear", self.env.nA]],
                            hidden_activations="relu", 
#                             output_activation="softmax", 
#                             dropout=0.5,
                            initialiser="xavier", 
                            batch_norm=False)
#         self.main_dqn = RNN(input_dim=2, layers_info=[["lstm", 16], ["linear", self.env.nA]],
#             hidden_activations="relu",
#             batch_norm=False, dropout=0.0, initialiser="xavier")


        self.target_dqn = CNN(input_dim=(self.env.num_state_channels, self.env.board_size_x, self.env.board_size_y), 
                            layers_info=[
                            ["conv", self.env.num_state_channels, 2, 1, 1], 
                            ["linear", 64],
                            ["linear", 16],
                            ["linear", self.env.nA]],
                            hidden_activations="relu", 
#                             output_activation="softmax", 
#                             dropout=0.5,
                            initialiser="xavier", 
                            batch_norm=False)
    
        # Send models to GPU
        self.main_dqn.to(self.device)
        self.target_dqn.to(self.device)
        
         # Optimizer and Loss function
        self.optimizer = torch.optim.Adam(self.main_dqn.parameters(), lr=1e-4)
        self.mse = torch.nn.MSELoss()
        self.L1 = torch.nn.SmoothL1Loss()
        
#         self.list_number_of_actions = pd.DataFrame (columns = ['episode','length'])
#         self.list_cumulative_loss = pd.DataFrame (columns = ['episode','loss'])
#         self.list_number_of_actions.length = self.list_number_of_actions['length'].astype(float)
#         self.list_cumulative_loss.length = self.list_cumulative_loss['loss'].astype(float)

    def select_action(self, state):
        """
          Return an action to take based on epsilon (greedy or random action)
          :param state: the current state
          :return action: next action to take
        """
        random_number = np.random.uniform()
        if random_number < self.epsilon:
            # Random action
            if self.env.env_name == "segment":
                return torch.tensor([random.randint(0,self.env.num_action_channels-1), random.randint(0,self.env.board_size_x-1), random.randint(0,self.env.board_size_y-1)])
            elif self.env.env_name == "battleship":
                return torch.tensor([random.randint(0,self.env.board_size_x-1), random.randint(0,self.env.board_size_y-1)])
        else:
            # Greedy action
            state = [state]
            state = torch.stack(state)
            state = state.to(self.device, dtype=torch.float)
            argmax = torch.argmax(self.main_dqn(state)).item()
            
            if self.env.env_name == "segment":
                return torch.tensor(np.unravel_index(argmax, (self.env.num_action_channels, self.env.board_size_x, self.env.board_size_y)))
            elif self.env.env_name == "battleship":
                return torch.tensor(np.unravel_index(argmax, (self.env.board_size_x, self.env.board_size_y)))
            
            
       
    def train(self):
        """
        Train the network with a batch of samples
        :param states: The state before taking the action
        :param actions: action taken
        :param rewards: Reward for taking that action
        :param next_states: The state that the agent enters after taking the action
        :return loss: the loss value after training the batch of samples
        """
        if len(self.buffer) >= self.batch_size:
            with torch.no_grad():
                states, actions, rewards, next_states, dones = self.buffer.sample(self.batch_size)
                        
            # Send data to GPU
            states = torch.stack(states).to(self.device, dtype=torch.float) 
            actions = torch.stack(actions).to(self.device, dtype=torch.float)
            rewards = torch.stack(rewards).to(self.device, dtype=torch.float)
            rewards = torch.reshape(rewards, (self.batch_size, 1))
            
            next_states = torch.stack(next_states).to(self.device, dtype=torch.float)
            dones = torch.stack(dones).to(self.device, dtype=torch.float)
            
            #TODO
            if self.env.env_name == "battleship":
                ravel = torch.tensor([[self.env.board_size_y*1.0], [1.0]], dtype=torch.float64).to(self.device, dtype=torch.float)
            elif self.env.env_name == "segment":        
                ravel = torch.tensor([[self.env.board_size_x * self.env.board_size_y * 1.0], [self.env.board_size_y*1.0], [1.0]], dtype=torch.float64).to(self.device, dtype=torch.float)

            actions = torch.matmul(actions,ravel) # size([128, 1])    

            # Calculate target Q values using the Target Network
            selection = torch.argmax(self.main_dqn(next_states), dim = 1).unsqueeze(1)
            
            evaluation = self.target_dqn(next_states)
            evaluation = evaluation.gather(1, selection.long()) #size [256,1]
#             evaluation = torch.reshape(evaluation, (-1,))
            
            
        
            #Create Done mask
            nonzero_indices = torch.nonzero(dones).reshape(-1).tolist()  
            dones_mask = torch.eye(self.batch_size)
            for index in nonzero_indices:
                dones_mask[index,index] = 0
            dones_mask = dones_mask.to(self.device, dtype=torch.float)

            # Calculte target
#             print("torch.matmul(dones_mask, evaluation*self.gamma)size =", torch.matmul(dones_mask, evaluation*self.gamma).size())
#             print("rewards size=", rewards.size())
            target = rewards + torch.matmul(dones_mask, evaluation*self.gamma)
            target = target.detach()
#             print("target size=", target.size())
#             print("target =", target)

            # Calculate Q values using the Main Network        
            n_classes = self.env.num_action_channels * self.env.board_size_x * self.env.board_size_y
            n_samples = self.batch_size
            labels = torch.flatten(actions.type(torch.LongTensor), start_dim=0)
            labels_tensor = torch.as_tensor(labels)
            action_masks = torch.nn.functional.one_hot(labels_tensor, num_classes=n_classes).to(self.device, dtype=torch.float)
            
            q_value = action_masks * self.main_dqn(states)
#             print("q-value size=",q_value.size())
#             print("q-value=",q_value)
            
            q_value = torch.sum(q_value, dim=-1).reshape((self.batch_size, 1))
#             print("q-value after size=",q_value.size())
#             print("q-value after=",q_value)
            
           
            # Calculate loss
            loss = self.mse(target, q_value)
#             loss = self.L1(target, q_value)

            # Optimize the model
            self.optimizer.zero_grad()
            loss.backward()
#             for param in self.main_dqn.parameters():
#                 # Clip the target to avoid exploding gradients
#                 param.grad.data.clamp_(-1e-6,1e-6)
            self.optimizer.step()
            
            # Soft Copy the Main Network's weights to the Target Network 
            self.soft_update_of_target_network(self.main_dqn, self.target_dqn,tau=1e-3)

            return loss
        return 0

# SACD AGENT

In [ ]:

            
class SAC_Discrete(Base_Agent):
    '''
    Soft Actor Critic for discrete action space
    '''
    def __init__(self, agent_name, env, gamma, batch_size, automatic_entropy_tuning):
        super().__init__(agent_name, env)        
       
        self.gamma = gamma # The weight for future rewards
        self.batch_size = batch_size
        self.automatic_entropy_tuning = automatic_entropy_tuning
        self.buffer = Replay()
        self.L1loss = torch.nn.SmoothL1Loss()
        self.mse = torch.nn.MSELoss()
        
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        
        if self.automatic_entropy_tuning:
            # we set the max possible entropy as the target entropy
            self.target_entropy = -np.log((1.0 / self.env.nA)) * 0.98
#             self.log_alpha = torch.zeros(1, requires_grad=True, device=self.device)
#             self.alpha = self.log_alpha.exp()
            self.alpha = torch.ones(1, requires_grad=True, device=self.device)
            self.alpha_optim = torch.optim.Adam([self.alpha], lr=1e-4, eps=1e-4)
        else:
            self.alpha = 0.2
            self.add_extra_noise = False
            self.do_evaluation_iterations = False
        
        
        self.critic_local = CNN(input_dim=(self.env.num_state_channels, self.env.board_size_x, self.env.board_size_y), layers_info=[
                            ["conv", self.env.num_state_channels, 2, 1, 1], 
                            ["linear", 128],
                            ["linear", 16],
                            ["linear", self.env.nA]],
                            hidden_activations="relu", 
#                             output_activation="softmax", 
#                             dropout=0.5,
                            initialiser="xavier", 
                            batch_norm=False)
        self.critic_target = CNN(input_dim=(self.env.num_state_channels, self.env.board_size_x, self.env.board_size_y), layers_info=[
                            ["conv", self.env.num_state_channels, 2, 1, 1], 
                            ["linear", 128],
                            ["linear", 16],
                            ["linear", self.env.nA]],
                            hidden_activations="relu", 
#                             output_activation="softmax", dropout=0.0,
                            initialiser="xavier", 
                            batch_norm=False)
        self.critic_local_2 = CNN(input_dim=(self.env.num_state_channels, self.env.board_size_x, self.env.board_size_y), layers_info=[
                            ["conv", self.env.num_state_channels, 2, 1, 1], 
                            ["linear", 128],
                            ["linear", 16],
                            ["linear", self.env.nA]],
                            hidden_activations="relu", 
#                             output_activation="softmax", dropout=0.0,
                            initialiser="xavier", 
                            batch_norm=False)
        self.critic_target_2 = CNN(input_dim=(self.env.num_state_channels, self.env.board_size_x, self.env.board_size_y), layers_info=[
                            ["conv", self.env.num_state_channels, 2, 1, 1], 
                            ["linear", 128],
                            ["linear", 16],
                            ["linear", self.env.nA]],
                            hidden_activations="relu", 
#                             output_activation="softmax", dropout=0.0,
                            initialiser="xavier", 
                            batch_norm=False)
        self.actor_local = CNN(input_dim=(self.env.num_state_channels, self.env.board_size_x, self.env.board_size_y), layers_info=[
                            ["conv", self.env.num_state_channels, 2, 1, 1], 
                            ["linear", 128],
                            ["linear", 16],
                            ["linear", self.env.nA]],
                            hidden_activations="relu", 
                            output_activation="softmax", 
#                             dropout=0.0,
                            initialiser="xavier", 
                            batch_norm=False)
        
        self.critic_local.to(self.device)
        self.critic_target.to(self.device)
        self.critic_local_2.to(self.device)
        self.critic_target_2.to(self.device)
        self.actor_local.to(self.device)
        
        self.actor_optimizer = torch.optim.Adam(self.actor_local.parameters(), lr=1e-4)
        self.critic_optimizer = torch.optim.Adam(self.critic_local.parameters(), lr=1e-4)
        self.critic_optimizer_2 = torch.optim.Adam(self.critic_local_2.parameters(), lr=1e-4)
        
        copy_model_over(self.critic_local, self.critic_target)
        copy_model_over(self.critic_local_2, self.critic_target_2)

    def create_actor_distribution(self, action_probabilities):
        """Creates a distribution that the actor can then use to randomly draw actions"""
        
        action_distribution = Categorical(action_probabilities)  # this creates a distribution to sample from
        return action_distribution


    def produce_action_and_action_info(self, state):
        '''
        Given the state, produces an action, the probability of the action, the log probability of the action, and
        the argmax action
        '''
#         print("state size=", state.size())
        action_probabilities = self.actor_local(state)
        max_probability_action = torch.argmax(action_probabilities, dim=-1)
        action_distribution = self.create_actor_distribution(action_probabilities)
        
        action = action_distribution.sample().cpu()
        # Have to deal with situation of 0.0 probabilities because we can't do log 0
        z = action_probabilities == 0.0
        z = z.float() * 1e-8
        log_action_probabilities = torch.log(action_probabilities + z)
        return action, (action_probabilities, log_action_probabilities), max_probability_action

    def calculate_critic_losses(self, state_batch, action_batch, reward_batch, next_state_batch, done_batch):
        '''
        Calculates the losses for the two critics. This is the ordinary Q-learning loss except the additional entropy
        term is taken into account
        '''
        with torch.no_grad():
            if self.env.env_name == "battleship":
                ravel = torch.tensor([[self.env.board_size_y*1.0], [1.0]], dtype=torch.float64).to(self.device, dtype=torch.float)
            elif self.env.env_name == "segment":
                ravel = torch.tensor([[self.env.board_size_x * self.env.board_size_y * 1.0], [self.env.board_size_y*1.0], [1.0]], dtype=torch.float64).to(self.device, dtype=torch.float)
                
            action_batch = torch.matmul(action_batch,ravel)
            
            next_state_action, (action_probabilities, log_action_probabilities), _ = self.produce_action_and_action_info(next_state_batch)
            
            qf1_next_target = self.critic_target(next_state_batch)
            qf2_next_target = self.critic_target_2(next_state_batch)
           
#             print("qf1_next_target=", qf1_next_target)
#             print("qf2_next_target=", qf2_next_target)
            
#             print("torch.min(qf1_next_target, qf2_next_target) size=", torch.min(qf1_next_target, qf2_next_target).size())
#             print("log_action_probabilities size", log_action_probabilities)
#             print("log_action_probabilities size", log_action_probabilities.size())
#             print("inside term size=", (torch.min(qf1_next_target, qf2_next_target) - self.alpha * log_action_probabilities))
            min_qf_next_target = action_probabilities * (torch.min(qf1_next_target, qf2_next_target) - self.alpha * log_action_probabilities)
            
#             print("min_qf_next_target before =", min_qf_next_target)
            min_qf_next_target = min_qf_next_target.sum(dim=1).unsqueeze(-1)
            
#             print("min_qf_next_target after =", min_qf_next_target)
            
            next_q_value = reward_batch + torch.matmul(done_batch, self.gamma * min_qf_next_target)

        
        qf1 = self.critic_local(state_batch).gather(1, action_batch.long())
        qf2 = self.critic_local_2(state_batch).gather(1, action_batch.long())
        
#         print("qf1  size =", qf1)
#         print("qf2  size =", qf2)
#         print("next_q_value  size =", next_q_value)
        
        qf1_loss = self.mse(qf1, next_q_value)
        qf2_loss = self.mse(qf2, next_q_value)

        return qf1_loss, qf2_loss

    def calculate_actor_loss(self, state_batch):
        '''
        Calculates the loss for the actor. This loss includes the additional entropy term
        '''
        _, (action_probabilities, log_action_probabilities), _ = self.produce_action_and_action_info(state_batch)
        
        qf1_pi = self.critic_local(state_batch)
        qf2_pi = self.critic_local_2(state_batch)
        min_qf_pi = torch.min(qf1_pi, qf2_pi) #size [256, 9]
        
#         print("min_qf_pi size = ", min_qf_pi)
#         print("log_action_probabilities=",log_action_probabilities)
        inside_term = self.alpha * log_action_probabilities - min_qf_pi   #size [256, 9]
#         print("inside_term size=", inside_term)
        
        entropies = torch.sum(log_action_probabilities * action_probabilities, dim=1)
        
        policy_loss = (action_probabilities * inside_term).sum(dim=1).mean()
        
        
#         # Expectations of entropies.
#         entropies = -torch.sum(action_probabilities * log_action_probabilities, dim=1, keepdim=True)

#         # Expectations of Q.
#         q = torch.sum(torch.min(qf1_pi, qf2_pi) * action_probabilities, dim=1, keepdim=True)

#         # Policy objective is maximization of (Q + alpha * entropy) with
#         # priority weights.
#         policy_loss = (- q - self.alpha * entropies).mean()

        return policy_loss, entropies
    
    

    def select_action(self, state):
        state = [state]
        state = torch.stack(state)
        state = state.to(self.device, dtype=torch.float)
        action, _, _ = self.produce_action_and_action_info(state)
        action = action #.detach().clone()
        
        if self.env.env_name == "battleship":
            return torch.tensor(np.unravel_index(action.item(), (self.env.board_size_x, self.env.board_size_y)))
        elif self.env.env_name == "segment":
            return torch.tensor(np.unravel_index(action.item(), (self.env.num_action_channels, self.env.board_size_x, self.env.board_size_y)))
        

    def take_optimisation_step(self, optimizer, network, loss, clipping_norm=None, retain_graph=False):
        """Takes an optimisation step by calculating gradients given the loss and then updating the parameters"""
        if not isinstance(network, list): network = [network]
        with torch.autograd.set_detect_anomaly(True):
            optimizer.zero_grad()
            loss.backward(retain_graph=retain_graph)
#             for net in network:
#                 for param in net.parameters():
#     #                 # Clip the target to avoid exploding gradients
#                     param.grad.data.clamp_(-1e-6,1e-6)
    
            if clipping_norm is not None:
                for net in network:
                    torch.nn.utils.clip_grad_norm_(net.parameters(), clipping_norm) #clip gradients to help stabilise training
#             optimizer.step()

    def calculate_entropy_tuning_loss(self, log_pi):
        """Calculates the loss for the entropy temperature parameter. This is only relevant if self.automatic_entropy_tuning
        is True."""
#         print("log_pi=",log_pi)
#         print("self.log_alpha=",self.log_alpha)
#         alpha_loss = -(self.log_alpha * (log_pi + self.target_entropy).detach()).mean()
        alpha_loss = -(self.alpha * (log_pi + self.target_entropy).detach()).mean()
#         print("alpha_loss=", alpha_loss)
        return alpha_loss

    def train(self):
        """
        Train the network with a batch of samples
        :param states: The state before taking the action
        :param actions: action taken
        :param rewards: Reward for taking that action
        :param next_states: The state that the agent enters after taking the action
        :return loss: the loss value after training the batch of samples
        """
        if len(self.buffer) >= self.batch_size:
            with torch.no_grad():
                states, actions, rewards, next_states, dones = self.buffer.sample(self.batch_size)
                states = torch.stack(states).to(self.device, dtype=torch.float)
                actions = torch.stack(actions).to(self.device, dtype=torch.float)
                rewards = torch.stack(rewards).to(self.device, dtype=torch.float)
                next_states = torch.stack(next_states).to(self.device, dtype=torch.float)
                rewards = torch.reshape(rewards, (self.batch_size, 1))

                dones = torch.stack(dones).to(self.device, dtype=torch.long)
                nonzero_indices = torch.nonzero(dones).reshape(-1).tolist()  
                dones_mask = torch.eye(self.batch_size)
                for index in nonzero_indices:
                    dones_mask[index,index] = 0
                dones_mask = dones_mask.to(self.device, dtype=torch.float)  # size [256, 256]
        
            #Updates the parameters for both critics
            qf1_loss, qf2_loss = self.calculate_critic_losses(states, actions, rewards, next_states, dones_mask)
            self.take_optimisation_step(self.critic_optimizer, self.critic_local, qf1_loss,None)
            self.take_optimisation_step(self.critic_optimizer_2, self.critic_local_2, qf2_loss,None)
            self.critic_optimizer.step()
            self.critic_optimizer_2.step()
            self.soft_update_of_target_network(self.critic_local, self.critic_target,tau=1e-3)
            self.soft_update_of_target_network(self.critic_local_2, self.critic_target_2,tau=1e-3)

            # update_actor_parameters
            policy_loss, log_pi = self.calculate_actor_loss(states)
            self.take_optimisation_step(self.actor_optimizer, self.actor_local, policy_loss,None)
            self.actor_optimizer.step()

            if self.automatic_entropy_tuning:
                alpha_loss = self.calculate_entropy_tuning_loss(log_pi)
            else: 
                alpha_loss = None
            
            
            if alpha_loss is not None:
                self.take_optimisation_step(self.alpha_optim, None, alpha_loss, None)
                self.alpha_optim.step()
#                 self.alpha = self.log_alpha.exp()

#         return policy_loss, qf1_loss, qf2_loss
            print("policy_loss = " + str(policy_loss.item()) + "| qf1_loss = " + str(qf1_loss.item()) + "| qf2_loss = " + str(qf2_loss.item()))
            return qf1_loss
        return 0

# Run Segment Game

In [ ]:
# #### SEGMENT EXPERIMENTS #####

env = SegmentEnv(max_steps=10, window=1, segment_length=10, noise=0.0, free_location=False, exploration_cost=1, pred_reward=1) #exploration_cost=0.05
env.reset(NEXT=True)

agent_random = Random_Agent(agent_name="random_agent", env=env)
agent_ddqn = DDQN(agent_name="ddqn_agent", env=env, epsilon=0.99, epsilon_decay_rate=0.999, gamma=0.99, batch_size=256)
agent_sacd = SAC_Discrete(agent_name="sacd_agent", env=env, gamma=0.99, batch_size=256, automatic_entropy_tuning=False)

agent_ddqn.runner(10000, training=True)
agent_sacd.runner(10000, training=True)
agent_random.runner(10000)


In [ ]:
results = []
results.append(agent_random.list_rewards)
results.append(agent_ddqn.list_rewards)
results.append(agent_sacd.list_rewards)
# results.append(agent_battleship_optimal.list_rewards)
plot_results(results, rolling = 200, download=True, filename = "segment")

In [ ]:
# def optimal_reward(segment_length, reward_pred, exploration_loss):
#     state_value = np.zeros(segment_length + 1)
#     state_value[1] = reward_pred
#     for i in range(2, segment_length + 1):
#         exploration_value = [state_value[k]*k/i + (1-k/i)*state_value[i-k] - exploration_loss for k in range(1, i)]
#         state_value[i] = max(reward_pred/i, np.max(exploration_value))
#     return state_value
# s = optimal_reward(segment_length=5, reward_pred=1, exploration_loss=1)

# Run Battleship game

In [ ]:
##### BATTLE SHIP EXPERIMENTS #####

env = BatteshipEnv(board_size_x=3, board_size_y=3, num_small_ships=1, num_medium_ships=1, num_large_ships=1, max_steps=6)

agent_ddqn = DDQN(agent_name="ddqn_agent", env=env, epsilon=0.99, epsilon_decay_rate=0.999, gamma=0.99, batch_size=256)
agent_sacd = SAC_Discrete(agent_name="sacd_agent", env=env, gamma=0.99, batch_size=256, automatic_entropy_tuning=False)
agent_battleship_optimal = Battleship_Baseline(agent_name="battleship_optimal_agent", env=env)
agent_random = Random_Agent(agent_name="random_agent", env=env)

agent_ddqn.runner(10000, training=True)
agent_sacd.runner(10000, training=True)
agent_battleship_optimal.runner(10000)
agent_random.runner(10000)



In [ ]:
results = []
results.append(agent_random.list_rewards)
results.append(agent_ddqn.list_rewards)
results.append(agent_sacd.list_rewards)
results.append(agent_battleship_optimal.list_rewards)
plot_results(results, rolling = 200, download=True, filename = "battleship")